## 고급주제

### 내용
1. Transfer Learning
    - 사전 학습된 모델을 새로운 문제에 적용
    - 적은 데이터로도 높은 성능을 얻을 수 있는 방법
    - 사전 학습된 모델 사용 - torchvision.models 모듈에서 사전 학습된 모델 로드

In [1]:
import torch
import torchvision.models as models

In [2]:
# Pretrained ResNet 모델 로드
resnet = models.resnet18(pretrained=True)

# 마지막 레이어 변경 (예: 10개의 클래스를 가진 새로운 데이터셋에 맞게)
num_features = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_features, 10)

# 특정 레이어의 가중치 고정
for param in resnet.parameters():
    param.requires_grad = False

c:\Users\perso\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\perso\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2. Fine-Tuning
    - 사전 학습된 모델의 일부 또는 전체를 재훈련하여 성능 개선
    - 모든 레이어의 가중치를 학습하도록 설정하거나, 일부 레이어만 학습 가능하도록 설정

In [3]:
# 특정 레이어만 학습 가능하도록 설정
for param in resnet.layer4.parameters():
    param.requires_grad = True

# 옵티마이저 설정 시 학습 가능한 파라미터만 전달
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

3. 예제 프로젝트 설명 및 진행
    - 개요
        - Kaggle 데이터셋을 이용한 실전 프로젝트 수행
        - 모델 설계, 훈련, 평가 및 결과 발표

    - 데이터셋 - CIFAR-10, Dogs vs. Cats 등 Kaggle 또는 기타 오픈 데이터셋에서 프로젝트 데이터셋 선택
    - 프로젝트 단계
        1. 데이터 전처리 및 준비
        2. 모델 설계 및 구현
        3. 모델 훈련 및 검증
        4. 성능 평가 및 결과 분석

4. 프로젝트 예시
    - Dogs vs. Cats 이미지 분류
        1. 데이터셋 로드 및 전처리
        2. Transfer Learning을 사용한 모델 구현
        3. 모델 훈련 및 평가

In [4]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [5]:
# 데이터 전처리 변환
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 로드
train_dataset = datasets.ImageFolder(root='./data/dogs-vs-cats/train/', transform=transform)
val_dataset = datasets.ImageFolder(root='./data/dogs-vs-cats/train/', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Pretrained ResNet 모델 로드 및 수정
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# 특정 레이어의 가중치 고정
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

In [6]:
# 모델 학습
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

Epoch 1, Loss: 0.12488087281510901
Epoch 2, Loss: 0.07997049633211092
Epoch 3, Loss: 0.07888865981654733
Epoch 4, Loss: 0.07412404753595277
Epoch 5, Loss: 0.0728756112084948


- 대략 47분 소요

In [7]:
# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 97.544%


5. 프로젝트 수행
    - 주어진 데이터셋을 사용하여 모델을 설계하고 훈련
    - Transfer Learning과 Fine-Tuning을 적용하여 성능 향상 시도
    - 모델의 성능을 검증 데이터셋을 통해 평가